In [54]:
# Importer les modules 
import scipy.stats as stats
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.interpolate import CubicSpline


# Définir les paramètres de la distribution normale (moyenne et écart-type)
moyenne = 0
ecart_type = 1

**BS sans dividendes**

In [55]:
# Prix du call et du put Black Scholes à t = 0

def call_BS (T,S0,K,sigma,r):
    d_1 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) + (1/2)*sigma*np.sqrt(T)
    d_2 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) - (1/2)*sigma*np.sqrt(T)
    cdf_value_d1 = stats.norm.cdf(d_1, loc=moyenne, scale=ecart_type)
    cdf_value_d2 = stats.norm.cdf(d_2, loc=moyenne, scale=ecart_type)
    return S0 * cdf_value_d1 - K*np.exp(- r*T) * cdf_value_d2

def put_BS (T,S0,K,sigma,r):
    d_1 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) + (1/2)*sigma*np.sqrt(T)
    d_2 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) - (1/2)*sigma*np.sqrt(T)
    cdf_value_d1 = stats.norm.cdf(d_1, loc=moyenne, scale=ecart_type)
    cdf_value_d2 = stats.norm.cdf(d_2, loc=moyenne, scale=ecart_type)
    oppose_cdf_value_d1 = - cdf_value_d1
    oppose_cdf_value_d2 = - cdf_value_d2
    return K*np.exp(- r*T) * oppose_cdf_value_d2 -  S0 * oppose_cdf_value_d1

# Greeks 

# Delta du call et du put sont différents 
def delta_call_BS (T,S0,K,sigma,r):
    d_1 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) + (1/2)*sigma*np.sqrt(T)
    d_2 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) - (1/2)*sigma*np.sqrt(T)
    cdf_value_d1 = stats.norm.cdf(d_1, loc=moyenne, scale=ecart_type)
    return cdf_value_d1

def delta_put_BS (T,S0,K,sigma,r):
    return delta_call_BS (T,S0,K,sigma,r) - 1
    
# Gamma du call et du put sont égaux
def gamma_BS (T,S0,K,sigma,r):
    d_1 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) + (1/2)*sigma*np.sqrt(T)
    return (1/(S0*sigma*np.sqrt(T)))*(1/np.sqrt(2*np.pi))*np.exp(-((d_1**2)/2))

# Vega du call et du put
def vega_BS (T,S0,K,sigma,r):
    d_1 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) + (1/2)*sigma*np.sqrt(T)
    return S0 * np.sqrt(T) * (1/np.sqrt(2*np.pi)) * np.exp(-((d_1**2)/2))


# Theta du call et du put sont différents    
def theta_call_BS (T,S0,K,sigma,r): 
    d_1 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) + (1/2)*sigma*np.sqrt(T)
    d_2 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) - (1/2)*sigma*np.sqrt(T)
    cdf_value_d1 = stats.norm.cdf(d_1, loc=moyenne, scale=ecart_type)
    cdf_value_d2 = stats.norm.cdf(d_2, loc=moyenne, scale=ecart_type)
    return ((S0 * sigma * np.exp(-((d_1)**2)/2))/(2*np.sqrt(2*(np.pi)*T))) + (r*K*np.exp(- r*T)*cdf_value_d2)

def theta_put_BS (T,S0,K,sigma,r): 
    d_1 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) + (1/2)*sigma*np.sqrt(T)
    d_2 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) - (1/2)*sigma*np.sqrt(T)
    cdf_value_d1 = stats.norm.cdf(d_1, loc=moyenne, scale=ecart_type)
    cdf_value_d2 = stats.norm.cdf(d_2, loc=moyenne, scale=ecart_type)
    return ((S0 * sigma * np.exp(-((d_1)**2)/2))/(2*np.sqrt(2*(np.pi)*T))) - (r*K*np.exp(- r*T)*(1 - cdf_value_d2))
    
    
    
    



**BS avec dividendes (à taux continu q)**

In [56]:
# Black-Scholes avec dividendes (versement des dividendes au taux continu q)

# Option call européenne (prix à l'instant 0)

def call_BS_dividend (T,S0,K,sigma,r,q): 
    d_1_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) + (1/2)*sigma*np.sqrt(T)
    d_2_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) - (1/2)*sigma*np.sqrt(T)
    cdf_value_d1_dividend = stats.norm.cdf(d_1_dividend, loc=moyenne, scale=ecart_type)
    cdf_value_d2_dividend = stats.norm.cdf(d_2_dividend, loc=moyenne, scale=ecart_type)
    return S0*np.exp(-q*T)*cdf_value_d1_dividend - K*np.exp(-r*T)*cdf_value_d2_dividend 

# option put européenne (prix à l'instant 0)

def put_BS_dividend (T,S0,K,sigma,r,q): 
    d_1_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) + (1/2)*sigma*np.sqrt(T)
    d_2_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) - (1/2)*sigma*np.sqrt(T)
    cdf_value_d1_dividend = stats.norm.cdf(d_1_dividend, loc=moyenne, scale=ecart_type)
    cdf_value_d2_dividend = stats.norm.cdf(d_2_dividend, loc=moyenne, scale=ecart_type)
    return K*np.exp(-r*T)*cdf_value_d1_dividend - S0*np.exp(-q*T)*cdf_value_d2_dividend


# Greeks (Modèle BS avec dividendes)

# Delta 

def delta_call_BS_dividendes (T,S0,K,sigma,r,q):
    d_1_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) + (1/2)*sigma*np.sqrt(T)
    cdf_value_d1_dividend = stats.norm.cdf(d_1_dividend, loc=moyenne, scale=ecart_type)
    return np.exp(-q*T)*cdf_value_d1_dividend

def delta_put_BS_dividendes (T,S0,K,sigma,r,q):
    return delta_call_BS_dividendes (T,S0,K,sigma,r,q) - np.exp(-q*T)

# Gamma du call et du put (même valeur)

def gamma_BS_dividendes (T,S0,K,sigma,r,q):
    d_1_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) + (1/2)*sigma*np.sqrt(T)
    return (1/(S0*sigma*np.sqrt(T)))*(1/np.sqrt(2*np.pi))*np.exp(-((d_1_dividend**2)/2))*np.exp(-q*T)
    

# Vega du call et du put (même valeur)

def vega_BS_dividende (T,S0,K,sigma,r,q):
    d_1_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) + (1/2)*sigma*np.sqrt(T)
    return S0 * np.sqrt(T) * (1/np.sqrt(2*np.pi)) * np.exp(-((d_1_dividend**2)/2))*np.exp(-q*T)

# Rho du call et du put sont différents 

def rho_call_BS_dividendes (T,S0,K,sigma,r,q):
    d_1_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) + (1/2)*sigma*np.sqrt(T)
    d_2_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) - (1/2)*sigma*np.sqrt(T)
    cdf_value_d1_dividend = stats.norm.cdf(d_1_dividend, loc=moyenne, scale=ecart_type)
    cdf_value_d2_dividend = stats.norm.cdf(d_2_dividend, loc=moyenne, scale=ecart_type)
    return K*T*np.exp(-r*T)*cdf_value_d2_dividend


def rho_put_BS_dividendes (T,S0,K,sigma,r,q):
    d_1_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) + (1/2)*sigma*np.sqrt(T)
    d_2_dividend = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp((r-q)*T))) - (1/2)*sigma*np.sqrt(T)
    cdf_value_opposee_d2_dividend = stats.norm.cdf(d_2_dividend, loc=moyenne, scale=ecart_type)
    return - K*T*np.exp(-r*T)*cdf_value_opposee_d2_dividend

    
    


    


**Greeks in the BS model with dividends**

![Alt text](image.png)

**Arbre binomiale**

In [57]:
"""
# Arbre binomiale 

# European call option 

def arbre_binomial_n_etapes_call(S0, K, r, u, d, n):
    dt = 1 / n  # Division de l'unité de temps en n étapes

    # Calcul du facteur de probabilité neutre au risque
    q = (1 + r - d) / (u - d)

    # Création de l'arbre des prix de l'actif à n étapes
    stock_tree = np.zeros((n + 1, n + 1))
    for i in range(n + 1):
        for j in range(i + 1):
            stock_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)

    # Calcul des paiements à l'expiration de l'option
    option_tree = np.zeros((n + 1, n + 1))
    for i in range(n + 1):
        option_tree[i, n] = max(stock_tree[i, n] - K, 0)

    # Remontée de l'arbre pour évaluer l'option
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option_tree[j, i] = (q * option_tree[j, i + 1] + (1 - q) * option_tree[j + 1, i + 1]) / (1 + r)

    return option_tree[0, 0]




# European put option 

def arbre_binomial_n_etapes_put(S0, K, r, u, d, n):
    dt = 1 / n  # Division de l'unité de temps en n étapes

    # Calcul du facteur de probabilité neutre au risque
    q = (1 + r - d) / (u - d)

    # Création de l'arbre des prix de l'actif à n étapes
    stock_tree = np.zeros((n + 1, n + 1))
    for i in range(n + 1):
        for j in range(i + 1):
            stock_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)

    # Calcul des paiements à l'expiration de l'option (pour un "put")
    option_tree = np.zeros((n + 1, n + 1))
    for i in range(n + 1):
        option_tree[i, n] = max(K - stock_tree[i, n], 0)

    # Remontée de l'arbre pour évaluer l'option (pour un "put")
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option_tree[j, i] = (q * option_tree[j, i + 1] + (1 - q) * option_tree[j + 1, i + 1]) / (1 + r)

    return option_tree[0, 0]
"""

'\n# Arbre binomiale \n\n# European call option \n\ndef arbre_binomial_n_etapes_call(S0, K, r, u, d, n):\n    dt = 1 / n  # Division de l\'unité de temps en n étapes\n\n    # Calcul du facteur de probabilité neutre au risque\n    q = (1 + r - d) / (u - d)\n\n    # Création de l\'arbre des prix de l\'actif à n étapes\n    stock_tree = np.zeros((n + 1, n + 1))\n    for i in range(n + 1):\n        for j in range(i + 1):\n            stock_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)\n\n    # Calcul des paiements à l\'expiration de l\'option\n    option_tree = np.zeros((n + 1, n + 1))\n    for i in range(n + 1):\n        option_tree[i, n] = max(stock_tree[i, n] - K, 0)\n\n    # Remontée de l\'arbre pour évaluer l\'option\n    for i in range(n - 1, -1, -1):\n        for j in range(i + 1):\n            option_tree[j, i] = (q * option_tree[j, i + 1] + (1 - q) * option_tree[j + 1, i + 1]) / (1 + r)\n\n    return option_tree[0, 0]\n\n\n\n\n# European put option \n\ndef arbre_binomial_n_etapes_pu

In [58]:

# Binomial trees 

# European call 
def european_call_option_pricing_binomial_trees(S0, K, r, n, u, d):
    p = (1 + r - d) / (u - d)

    # Creation of the stock price tree
    stock_tree = np.zeros((n + 1, n + 1))
    stock_tree[0, 0] = S0
    for i in range(1, n + 1):
        stock_tree[i, 0] = stock_tree[i - 1, 0] * u
        for j in range(1, i + 1):
            stock_tree[i, j] = stock_tree[i - 1, j - 1] * d

    # Calculate option values at expiration
    option_tree = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        option_tree[n, j] = max(0, stock_tree[n, j] - K)

    # Backward induction for calculating the option's current value
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option_tree[i, j] = (p * option_tree[i + 1, j] + (1 - p) * option_tree[i + 1, j + 1]) / (1 + r)

    return option_tree[0, 0]

# European put
def european_put_option_pricing_binomial_trees(S0, K, r, n, u, d):
    p = (1 + r - d) / (u - d)

    # Creation of the stock price tree
    stock_tree = np.zeros((n + 1, n + 1))
    stock_tree[0, 0] = S0
    for i in range(1, n + 1):
        stock_tree[i, 0] = stock_tree[i - 1, 0] * u
        for j in range(1, i + 1):
            stock_tree[i, j] = stock_tree[i - 1, j - 1] * d

    # Calculate option values at expiration
    option_tree = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        option_tree[n, j] = max(0, K - stock_tree[n, j])

    # Backward induction for calculating the option's current value
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option_tree[i, j] = (p * option_tree[i + 1, j] + (1 - p) * option_tree[i + 1, j + 1]) / (1 + r)

    return option_tree[0, 0]


**Volatilité implicite**

In [59]:
"""
# Volatilité implicite BS sans dividendes (Méthode de Newton)

def vol_implicit_BS (T,S0,K,r,n,market_price):
    if n == 0 : 
        return np.sqrt((2*np.abs(np.log((S0*np.exp(r)*T)/K)))/T)
    elif n >= 1 :
        return vol_implicit_BS (T,S0,K,r,n-1,market_price)- ((call_BS (T,S0,K,vol_implicit_BS (T,S0,K,r,n-1,market_price),r)) - market_price)/(vega_BS (T,S0,K,vol_implicit_BS (T,S0,K,r,n-1,market_price),r))



# Volatilité implicite BS avec dividendes (Méthode de Newton)  

def vol_implicit_BS_dividende (T,S0,K,r,q,n,market_price):
    if n == 0 : 
        return np.sqrt((2*np.abs(np.log((S0*np.exp(r-q)*T)/K)))/T)
    elif n >= 1 :
        return vol_implicit_BS_dividende (T,S0,K,r,q,n-1,market_price)- ((call_BS_dividend(T,S0,K,vol_implicit_BS_dividende (T,S0,K,r,q,n-1,market_price),r,q)) - market_price)/(vega_BS_dividendes(T,S0,K,vol_implicit_BS_dividende (T,S0,K,r,q,n-1,market_price),r,q))

"""


'\n# Volatilité implicite BS sans dividendes (Méthode de Newton)\n\ndef vol_implicit_BS (T,S0,K,r,n,market_price):\n    if n == 0 : \n        return np.sqrt((2*np.abs(np.log((S0*np.exp(r)*T)/K)))/T)\n    elif n >= 1 :\n        return vol_implicit_BS (T,S0,K,r,n-1,market_price)- ((call_BS (T,S0,K,vol_implicit_BS (T,S0,K,r,n-1,market_price),r)) - market_price)/(vega_BS (T,S0,K,vol_implicit_BS (T,S0,K,r,n-1,market_price),r))\n\n\n\n# Volatilité implicite BS avec dividendes (Méthode de Newton)  \n\ndef vol_implicit_BS_dividende (T,S0,K,r,q,n,market_price):\n    if n == 0 : \n        return np.sqrt((2*np.abs(np.log((S0*np.exp(r-q)*T)/K)))/T)\n    elif n >= 1 :\n        return vol_implicit_BS_dividende (T,S0,K,r,q,n-1,market_price)- ((call_BS_dividend(T,S0,K,vol_implicit_BS_dividende (T,S0,K,r,q,n-1,market_price),r,q)) - market_price)/(vega_BS_dividendes(T,S0,K,vol_implicit_BS_dividende (T,S0,K,r,q,n-1,market_price),r,q))\n\n'

In [60]:
# Point de départ de l'algo de Newton pour calculer la vol implicite
def vol_implicit_BS_dividende_initiale(T, S0, K, r, q):
    vol = np.sqrt((2 * np.abs(np.log((S0 * np.exp((r - q) * T)) / K))) / T)
    return vol

In [61]:
def vega_BS_dividendes(T, S0, K, sigma, r, q):
    d1 = (np.log(S0 / K) + (r - q + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
    
    vega = S0 * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)
    
    return vega

def vol_implicit_BS_dividende(T, S0, K, r, q, market_price, max_iter=1000, tol=1e-5):
    # Initial guess for volatility based on the provided formula
    vol = np.sqrt((2 * np.abs(np.log((S0 * np.exp((r - q) * T)) / K))) / T)
    
    for _ in range(max_iter):
        d1 = (np.log(S0 / K) + (r - q + (vol ** 2) / 2) * T) / (vol * np.sqrt(T))
        d2 = d1 - vol * np.sqrt(T)
        option_price = S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        vega = vega_BS_dividendes(T, S0, K, vol, r, q)
        price_error = option_price - market_price
        
        if abs(price_error) < tol:
            return vol
        
        vol = vol - price_error / vega
    
    # If max_iter is reached without convergence, return None or raise an exception
    return None

In [62]:
def vega_DLN_dividendes(T, S0, K,a, sigma, r, q):
    d1 = (np.log((S0+a) / (K+a)) + (r - q + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
    
    vega = (S0+a) * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)
    
    return vega


# Si on a le market price 
def vol_implicit_DLN_dividende(T, S0, K,a, r, q, market_price, max_iter=1000, tol=1e-5):
    # Initial guess for volatility based on the provided formula
    vol = np.sqrt((2 * np.abs(np.log(((S0+a) * np.exp((r - q) * T)) / (K+a)))) / T)
    
    for _ in range(max_iter):
        d1 = (np.log((S0+a) / (K+a)) + (r - q + (vol ** 2) / 2) * T) / (vol * np.sqrt(T))
        d2 = d1 - vol * np.sqrt(T)
        option_price = (S0+a) * np.exp(-q * T) * norm.cdf(d1) - (K+a) * np.exp(-r * T) * norm.cdf(d2)
        vega = vega_DLN_dividendes(T, S0, K,a, vol, r, q)
        price_error = option_price - market_price
        
        if abs(price_error) < tol:
            return vol
        
        vol = vol - price_error / vega
    
    # If max_iter is reached without convergence, return None or raise an exception
    return None


# Si on n'a pas le market price (Relation avec la vol implicite BS)






**Alternatives models (DLN, Bachelier...)**

In [63]:
# Alternatives model 

# Displaced log normal 

# Prix d'un call 
def call_DLN (T,S0,K,a,sigma,r): 
    return call_BS (T, S0 + a, K + a, sigma, r)
# Prix d'un put 
def put_DLN (T,S0,K,a,sigma,r): 
    return put_BS (T, S0 + a, K + a, sigma, r)




# Bachelier 

# Prix d'un call
def call_Bachelier (T, S0, K, sigma, r) :
    value = (S0 - K)/ (sigma*np.sqrt(T))
    cdf_value = stats.norm.cdf(value, loc=moyenne, scale=ecart_type)
    return ((S0 - K)*cdf_value) + sigma*np.sqrt(T)*(1/np.sqrt(2*np.pi))*np.exp(-(value**2)/2)
    


**Pricing du power call**

In [64]:
def power_call_BS_dividend (T,S0,K,sigma,r,q,n): 
    d_1_dividend = (1/(sigma*np.sqrt(T))) * (np.log(S0/K**(1/n)) + (r-q + (n-0.5)*sigma**2)*T)
    d_2_dividend = d_1_dividend - n*sigma*np.sqrt(T)
    cdf_value_d1_dividend = stats.norm.cdf(d_1_dividend, loc=moyenne, scale=ecart_type)
    cdf_value_d2_dividend = stats.norm.cdf(d_2_dividend, loc=moyenne, scale=ecart_type)
    return (S0**n)*np.exp((n-1)*(r+(n*sigma**2)/2)*T- n*q*T)*cdf_value_d1_dividend - K*np.exp(-r*T)*cdf_value_d2_dividend 

**Option digitale (binaire)**

In [65]:
def call_digital_option (T,S0,K,sigma,r):
    d_2 = ((1/(sigma*np.sqrt(T)))*np.log((S0/K) * np.exp(r*T))) - (1/2)*sigma*np.sqrt(T)
    cdf_value_d2 = stats.norm.cdf(d_2, loc=moyenne, scale=ecart_type)
    return np.exp(- r*T) * cdf_value_d2

**Options américaines**

In [66]:
# American options 

# American call
def american_call_option_pricing(S0, K, r, n, u, d):
    p = (1 + r - d) / (u - d)

    # Création de l'arbre des prix de l'actif sous-jacent
    stock_tree = np.zeros((n + 1, n + 1))
    stock_tree[0, 0] = S0
    for i in range(1, n + 1):
        stock_tree[i, 0] = stock_tree[i - 1, 0] * u
        for j in range(1, i + 1):
            stock_tree[i, j] = stock_tree[i - 1, j - 1] * d

    # Calcul des valeurs d'option à l'expiration
    option_tree = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        option_tree[n, j] = max(0, stock_tree[n, j] - K)

    # Backward induction pour calculer la valeur actuelle de l'option
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option_tree[i, j] = (p * option_tree[i + 1, j] + (1 - p) * option_tree[i + 1, j + 1]) / (1 + r)
            stock_price = stock_tree[i, j]
            early_exercise = max(0, stock_price - K)
            option_tree[i, j] = max(option_tree[i, j], early_exercise)

    return option_tree[0, 0]

# American put 
def american_put_option_pricing(S0, K, r, n, u, d):
    p = (1 + r - d) / (u - d)

    # Création de l'arbre des prix de l'actif sous-jacent
    stock_tree = np.zeros((n + 1, n + 1))
    stock_tree[0, 0] = S0
    for i in range(1, n + 1):
        stock_tree[i, 0] = stock_tree[i - 1, 0] * u
        for j in range(1, i + 1):
            stock_tree[i, j] = stock_tree[i - 1, j - 1] * d

    # Calcul des valeurs d'option à l'expiration
    option_tree = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        option_tree[n, j] = max(0, K - stock_tree[n, j])

    # Backward induction pour calculer la valeur actuelle de l'option
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option_tree[i, j] = (p * option_tree[i + 1, j] + (1 - p) * option_tree[i + 1, j + 1])/(1+r)
            stock_price = stock_tree[i, j]
            early_exercise = max(0, K - stock_price)
            option_tree[i, j] = max(option_tree[i, j], early_exercise)

    return option_tree[0, 0]

In [67]:
# Aggrégation du prix 

# Volume Weighted Median

def compute_vwm(file_name):
    # Step 1: Load the data
    data = pd.read_csv(file_name, delimiter=';')
    
    # Step 2: Sort the data based on Price
    data = data.sort_values(by='Price')
    
    # Step 3: Calculate cumulative volumes
    data['Cumulative_Volume'] = data['Volume'].cumsum()
    total_volume = data['Volume'].sum()
    
    # Step 4: Find the interval where the median volume lies
    median_volume = total_volume / 2
    below_median = data[data['Cumulative_Volume'] < median_volume]
    above_median = data[data['Cumulative_Volume'] >= median_volume]

    if not below_median.empty:
        price_below = below_median['Price'].iloc[-1]
        volume_below = below_median['Cumulative_Volume'].iloc[-1]
    else:
        price_below = 0
        volume_below = 0

    price_above = above_median['Price'].iloc[0]
    volume_above = above_median['Cumulative_Volume'].iloc[0]
    
    # Step 5: Linearly interpolate to find the VWM
    vwm = price_below + (price_above - price_below) * (median_volume - volume_below) / (volume_above - volume_below)

    return round(vwm, 3)

In [82]:
# Volatilité d'un asset path dependant (IL FAUT PENSER A REMPLACER LES POIDS ET LES VOLATILITES)

np.sqrt((0.349000**2 + 0.334000**2 + 0.317000**2 + 3*(0.297000)**2 + 4*(0.269000)**2)/10)

0.297979361701444

* Between now and 1 months, it is equal to 0.349000
* Between 1 and 2 months, it is equal to 0.334000
* Between 2 and 3 months, it is equal to 0.317000
* Between 3 and 6 months, it is equal to 0.297000
* Between 6 and 10 months, it is equal to 0.269000

In [83]:
compute_vwm(file_name = "C:/Users/ayman/Downloads/téléchargement (38).csv")

23.729

In [84]:
# Question Delta-Gamme Hedging avec un deuxième call
gamma_BS (T=0.456000,S0=105.670000,K=100.650000,sigma=0.178000,r=0.003000)/ gamma_BS (T=0.674000,S0=105.670000,K=101.170000,sigma=0.178000,r=0.003000)

1.1686927088698669

In [71]:
# Corrélation spot (instannée) (EXEMPLE AVEC DES VECTEURS DE VOL DE DIM 2)

# (0.090000  0.111000)
# (-0.024000 0.049000)
# (0.090000*(-0.024000) + 0.111000*0.049000)/(np.sqrt(0.090000**2 + 0.111000**2)*np.sqrt(0.024000**2 + 0.049000**2))

In [85]:
(-0.075000*0.233000 + 0.209000*(-0.235000))/(np.sqrt(0.075000**2 + 0.209000**2)*np.sqrt(0.233000**2 + 0.235000**2))


-0.9062005225516588

In [88]:
sigma11 = np.sqrt(0.207000**2 - 0.112000**2)
sigma12 = np.sqrt(0.220000**2 - 0.124000**2)
correl = (sigma11*sigma12 - 0.112000*0.124000)/(0.207000*0.220000)
correl 

0.3897079885894204

In [86]:
np.sqrt (0.1593**2 + 0.1241**2 + 2*0.53*0.1241*0.1593)

0.24846041495578325

**IV DELTA 25**

In [74]:
# IV Delta 25

def calculate_iv_25(csv_path, forward, T):
    # Read the CSV data and drop the unwanted column
    data = pd.read_csv(csv_path, sep=";").drop(columns=["Unnamed: 2"])

    # Extract strike and implied volatility values
    strike = np.array(data["Strike"])
    implied_vol = np.array(data["IV"])

    # Compute moneyness and d1 values
    moneyness = np.log(strike / forward)
    d1 = -moneyness / (implied_vol * np.sqrt(T)) + 0.5 * implied_vol * np.sqrt(T)

    # Compute delta values and flip them
    delta = norm.cdf(d1)
    delta = np.flip(delta)
    
    # Flip implied volatility values
    implied_vol = np.flip(implied_vol)

    # Compute cubic spline interpolation
    cs = CubicSpline(delta, implied_vol)
    iv_25 = cs(0.25)

    return iv_25

**P&L**


In [75]:

def bs_call_price(S, K, T, r, vol):
    d1 = (np.log(S / K) + (r + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def bs_call_delta(S, K, T, r, vol):
    d1 = (np.log(S / K) + (r + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
    return norm.cdf(d1)

def pnl_delta_hedging(data_file, T, K, r, vol):
    data = pd.read_csv(data_file, sep=";")
    data['TimeToMaturity'] = T - data['Date']

    # Initialiser les variables
    option_initial_price = bs_call_price(data.iloc[0]['Spot'], K, data.iloc[0]['TimeToMaturity'], r, vol)
    positions = [-option_initial_price]
    delta_previous = 0
    cash = option_initial_price

    for i, row in data.iterrows():
        # Calculer le delta actuel
        delta_current = bs_call_delta(row['Spot'], K, row['TimeToMaturity'], r, vol)
        
        # Cash flow due to rebalancing
        dS = row['Spot'] * (delta_current - delta_previous)
        cash -= dS

        # Store the delta for next iteration
        delta_previous = delta_current
        
        # Interest on cash
        if i < len(data) - 1:
            dt = data.iloc[i+1]['Date'] - row['Date']
            cash *= np.exp(r * dt)

    # PnL at maturity
    call_payoff = max(data.iloc[-1]['Spot'] - K, 0)
    final_cash = cash - call_payoff + delta_previous * data.iloc[-1]['Spot']
    pnl = final_cash + positions[0]
    return pnl


In [76]:
def bs_call_price_2(S, K, T, r, vol):
    if T <= 0:  # Si on est à maturité ou au-delà
        return max(S-K, 0)
    
    d1 = (np.log(S / K) + (r + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def bs_call_delta_2(S, K, T, r, vol):
    if T <= 0:  # Si on est à maturité ou au-delà
        return 1.0 if S > K else 0.0
    
    d1 = (np.log(S / K) + (r + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
    return norm.cdf(d1)

def pnl_delta_hedging_2(data_file, T, K, r, vol):
    data = pd.read_csv(data_file, sep=";")
    data['TimeToMaturity'] = T - data['Date']

    # Initialiser les variables
    option_initial_price = bs_call_price_2(data.iloc[0]['Spot'], K, data.iloc[0]['TimeToMaturity'], r, vol)
    positions = [-option_initial_price]
    delta_previous = 0
    cash = option_initial_price

    for i, row in data.iterrows():
        # Calculer le delta actuel
        delta_current = bs_call_delta_2(row['Spot'], K, row['TimeToMaturity'], r, vol)
        
        # Cash flow due to rebalancing
        dS = row['Spot'] * (delta_current - delta_previous)
        cash -= dS

        # Store the delta for next iteration
        delta_previous = delta_current
        
        # Interest on cash
        if i < len(data) - 1:
            dt = data.iloc[i+1]['Date'] - row['Date']
            cash *= np.exp(r * dt)

    # PnL at maturity
    call_payoff = max(data.iloc[-1]['Spot'] - K, 0)
    final_cash = cash - call_payoff + delta_previous * data.iloc[-1]['Spot']
    pnl = final_cash + positions[0]
    return pnl

# pnl_delta_hedging_2(data_file = "C:/Users/ayman/Downloads/téléchargement (36).csv", T=0.620000, K=104.430000, r=0.006000, vol=0.231900)


In [81]:
american_put_option_pricing(S0=99.52, K=100.42, r=0.0075, n=9, u=1.0734, d=0.9316)

6.541679479122793

In [87]:
calculate_iv_25(csv_path = "C:/Users/ayman/Downloads/téléchargement (39).csv", forward = 109.213000, T = 0.541000 )

array(0.67331116)